<a href="https://colab.research.google.com/github/mra49/Capstone/blob/main/Test_code_for_capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Bitsy EDA's

In [ ]:
import pandas as pd

# Group by 'Item Description' and sum the 'Extended Price'
grouped_data = df_rfm.groupby('Item Description')['Extended Price'].sum().reset_index()

# Rename the columns for clarity
grouped_data.columns = ['Item Description', 'Total Extended Price']

#sort data
sorted_data = grouped_data.sort_values(by='Total Extended Price', ascending=False)

# Output the result
print(sorted_data.head())


                   Item Description  Total Extended Price
72            foamy dishwash 6000ml          2.172554e+06
102      foamy laundry power gel 5l          2.154178e+06
181  style apple blossom sh. 4000ml          1.300975e+06
226  style shampoo cool ocean 4000m          1.218185e+06
91     foamy general cleaner 6000ml          1.196581e+06


In [ ]:
# Group by 'Item Number' and filter out those that appear less than 10 times, then overwrite df
df = df.groupby('Item Number').filter(lambda x: len(x) >= 10)

#not eda's

In [ ]:
##############  Test sample for prev checkPoint: DF_FAULT CONTAINS TWO COLUMNS THE UNIQUE IDS AND THEIR DESCRIPTION BUT ONLY IN'S THAT SHARE THE DESC ##############################
import pandas as pd

# Assuming df_copy is your DataFrame and it's already loaded

# Group by 'Item Description' and count the unique 'Item Numbers' for each description
description_counts = df_copy.groupby('Item Description')['Item Number'].nunique()

# Filter to get descriptions that are associated with more than one 'Item Number'
duplicate_descriptions = description_counts[description_counts > 1].index.tolist()

# Create a DataFrame with 'Item Number' and 'Item Description' where the descriptions are duplicated
df_fault = df_copy[df_copy['Item Description'].isin(duplicate_descriptions)][['Item Number', 'Item Description']]

# Drop duplicates to avoid repeating the same 'Item Number' with the same 'Item Description'
df_fault = df_fault.drop_duplicates()
df_fault.to_csv("df_fault.csv", index = False)
print(df_fault)


       Item Number                Item Description
1            56240            style mask kiwi 500g
36           52439  reflet argent gris nacre shamp
43           56592  style bone marrow conditioning
58           56979  style bone marrow conditioning
63           55562  style shampoo apple blossom 22
...            ...                             ...
86462        76654  foamy dishwashing liquid lemon
91701        76656  foamy laundry power gel 3100ml
102626       76666  foamy floor cleaner 5.1l + foa
102640       76665  foamy laundry 5.1l + foamy fab
102641       76667  foamy floor cleaner 5.1l + foa

[74 rows x 2 columns]


In [ ]:
############   Testing the Unifying Item Numbers Method applied on df_copy   ############
import pandas as pd

# Assuming df_fault is your DataFrame and it's already loaded

# Sort df_fault by 'Item Description' and 'Item Number'
df_fault_sorted = df_fault.sort_values(by=['Item Description', 'Item Number'])

# Group by 'Item Description' and select the first 'Item Number' as the representative
# Add a zero at the beginning of each representative 'Item Number'
rep_item_numbers = df_fault_sorted.groupby('Item Description')['Item Number'].first().apply(lambda x: '0' + str(x))

# Create a mapping from 'Item Description' to the modified representative 'Item Number'
description_to_rep_item_number = rep_item_numbers.to_dict()

# Map the representative 'Item Number' back to the df_fault DataFrame in a new column 'Unified Description'
df_fault['Unified Description'] = df_fault['Item Description'].map(description_to_rep_item_number)

# df_fault now has an additional column 'Unified Description'


In [ ]:
import pandas as pd

# Assuming df_fault is your DataFrame and it's already loaded

# Count the number of unique item numbers in the 'Item Number' column
unique_item_numbers_count = df_fault['Item Number'].nunique()
print(f"Number of unique Item Numbers: {unique_item_numbers_count}")

# Count the number of unique item numbers in the 'Unified Description' column
unique_unified_numbers_count = df_fault['Unified Description'].nunique()
print(f"Number of unique Unified Descriptions: {unique_unified_numbers_count}")
print(df_fault.info())

Number of unique Item Numbers: 74
Number of unique Unified Descriptions: 31
<class 'pandas.core.frame.DataFrame'>
Int64Index: 74 entries, 1 to 102641
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Item Number          74 non-null     object
 1   Item Description     74 non-null     object
 2   Unified Description  74 non-null     object
dtypes: object(3)
memory usage: 2.3+ KB
None


In [ ]:
df_fault.to_csv("df_fault.csv", index = False)

In [ ]:
# Count how many 'No' values are in the 'Unique Description' column
non_unique_count = (df_copy['Unique Description'] == 'No').sum()
print(f"Number of item numbers with non-unique descriptions: {non_unique_count}")

#here there's a flow, while the yes and no column is important, it's hindering or making the process of calculating how many missings more comlicated

# Count the number of unique item descriptions in the 'Item Description' column
unique_descriptions_count = df_copy['Item Description'].nunique()
print(f"There are {unique_descriptions_count} unique item descriptions.")

unique_descriptions_count = df_copy['Item Number'].nunique()
print(f"There are {unique_descriptions_count} unique item numbers.")
print("This means that there are Item Numbers with the same Item Description")


Number of item numbers with non-unique descriptions: 0
There are 879 unique item descriptions.
There are 922 unique item numbers.
This means that there are Item Numbers with the same Item Description


#Price Cleaning

In [2]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Capstone/df_latest.csv")

In [4]:
df.columns

Index(['Location', 'Brand', 'SubBrand', 'Category', 'SOP Number',
       'Item Number', 'Item Description', 'QTY', 'Unit Price',
       'Extended Price', 'Customer Number', 'Document Date', 'Item Short Name',
       'Customer Name', 'Location ID', 'Offer Status', 'product_type',
       'product_scent', 'product_size', 'Description Length', 'Subcategory',
       'With Offer', 'Day', 'Month', 'Year'],
      dtype='object')

In [3]:
rate_df = pd.read_csv("/content/drive/MyDrive/Capstone/usd-lbp.csv")

In [4]:
import pandas as pd

# Convert the date-time column in rate_df to datetime and extract the date part
rate_df['Date'] = pd.to_datetime(rate_df['DateTime']).dt.date

# Aggregate rate_df to have only one rate per date if there are multiple entries per date
# Here, we are taking the mean of the rates for each date
daily_rates = rate_df.groupby('Date')['USD to LBP'].mean().reset_index()

# Convert the date column in df to datetime (if it's not already)
df['Document Date'] = pd.to_datetime(df['Document Date']).dt.date

# Merge the DataFrames on the date column
# This will add the mean daily rate to your main df
df_merged = pd.merge(df, daily_rates, left_on='Document Date', right_on='Date', how='left')

# Drop the extra 'Date' column from the merge if you wish
df_merged.drop('Date', axis=1, inplace=True)


In [8]:
df_merged.columns

Index(['Location', 'Brand', 'SubBrand', 'Category', 'SOP Number',
       'Item Number', 'Item Description', 'QTY', 'Unit Price',
       'Extended Price', 'Customer Number', 'Document Date', 'Item Short Name',
       'Customer Name', 'Location ID', 'Offer Status', 'product_type',
       'product_scent', 'product_size', 'Description Length', 'Subcategory',
       'With Offer', 'Day', 'Month', 'Year', 'USD to LBP'],
      dtype='object')

In [9]:
df = df_merged

In [9]:
# import pandas as pd


# # Ensure the DataFrame is sorted by date
# df.sort_values(by='Document Date', inplace=True)

# # Identify the top 10 selling product types
# top_product_types = df.groupby('product_type')['QTY'].sum().nlargest(10).index


# # Function to adjust price
# def adjust_price(row):
#     if row['product_type'] in top_product_types and row['Unit Price'] > 16:
#         # Filter to find prices for the same item on earlier dates, less than $25 but more than $2.5
#         previous_prices = df[(df['Document Date'] < row['Document Date']) &
#                              (df['Item Number'] == row['Item Number']) &
#                              (df['Unit Price'] < 16) &
#                              (df['Unit Price'] > 2.5)]

#         # Sort by date to get the closest previous price
#         previous_prices = previous_prices.sort_values(by='Document Date', ascending=False)

#         # Check if there are any such prices and return the closest one, else return the current price
#         if not previous_prices.empty:
#             return previous_prices.iloc[0]['Unit Price']
#         else:
#             return row['Unit Price']
#     else:
#         return row['Unit Price']

# # Apply the function to create the 'calculated price' column
# df['calculated price'] = df.apply(adjust_price, axis=1)

In [18]:
import os
os.chdir('/content/drive/MyDrive/Capstone')

In [10]:
df['USD to LBP'].head()

0    2118.333333
1    2118.333333
2    2118.333333
3    2118.333333
4    2118.333333
Name: USD to LBP, dtype: float64

In [6]:
df.shape

(101542, 25)

In [11]:
# Identify the top 10 selling product types
top_product_types = df.groupby('product_type')['QTY'].sum().nlargest(10).index

# Ensure that 'Unit Price' and 'USD to LBP' are numeric
df['Unit Price'] = pd.to_numeric(df['Unit Price'], errors='coerce')
df['USD to LBP'] = pd.to_numeric(df['USD to LBP'], errors='coerce')

# Replace 0 with 1 in 'USD to LBP' to avoid division by zero
df['USD to LBP'].replace(0, 1, inplace=True)
df['USD to LBP'].fillna(1, inplace=True) # Replace NaN with 1

# Function to calculate the adjusted rate
def calculate_rate(row):
    if row['product_type'] in top_product_types and row['Unit Price'] > 17 and row['USD to LBP'] != 1:
        return (row['Unit Price'] * 1500) / row['USD to LBP']
    else:
        return row['Unit Price']

# Apply the function to create the 'rate_calculated' column
df['rate_calculated'] = df.apply(calculate_rate, axis=1)


In [14]:
df.shape

(101542, 27)

In [15]:
df['rate_calculated'].head(35)

0      8.670000
1      0.000000
2     11.020000
3      2.098000
4      6.904000
5      0.788000
6      0.899000
7      0.400000
8      1.585000
9      8.670000
10     8.670000
11     8.670000
12    14.700000
13     0.000000
14     0.000000
15     0.000000
16     0.000000
17     0.000000
18    12.214792
19    12.214792
20    97.680000
21     1.498000
22     0.000000
23     0.000000
24     0.000000
25    14.700000
26    14.700000
27    14.700000
28    14.700000
29     3.297000
30    12.214792
31     0.000000
32     3.331000
33     0.000000
34     3.331000
Name: rate_calculated, dtype: float64

In [12]:
# Assuming 'df' is your DataFrame and it has columns 'product_type' and 'QTY'

# Calculate the total sales for each product type
total_sales_by_type = df.groupby('product_type')['QTY'].sum()

# Identify the top 10 selling product types
top_product_types = total_sales_by_type.nlargest(10)

# Calculate the total sales of the top 10 selling product types
total_sales_top_10 = top_product_types.sum()

# Calculate the total sales in the entire DataFrame
total_sales_overall = total_sales_by_type.sum()

# Calculate the proportion
proportion_top_10 = total_sales_top_10 / total_sales_overall

# Print the proportion
print(f"The top 10 selling product types represent {proportion_top_10:.2%} of the total amount sold.")


The top 10 selling product types represent 93.70% of the total amount sold.


In [17]:
top_product_types

product_type
shampoo              643253
dishwashing          334949
laundry gel          110772
general cleaner       81862
hand soap             63985
disinfectant          59659
multi-use shampoo     49957
fabric softner        33992
floor cleaner         22988
shower gel            16784
Name: QTY, dtype: int64

In [13]:
df.to_csv("df_prices_adjusted.csv", index = False)

In [27]:
import pandas as pd

# Assuming your DataFrame is named df and it's already loaded

# Convert 'Document Date' to datetime if it's not already
df['Document Date'] = pd.to_datetime(df['Document Date'])

# Extract day, month, and year
df['Day'] = df['Document Date'].dt.day
df['Month'] = df['Document Date'].dt.month
df['Year'] = df['Document Date'].dt.year

In [19]:
import pandas as pd

# Load your original DataFrame
df_invoice = df

# Group by 'SOP Number' and aggregate
invoice_df = df_invoice.groupby('SOP Number').agg({
    'Item Number': pd.Series.nunique,
    'Category': pd.Series.nunique,
    'Subcategory': pd.Series.nunique,  # Assuming 'SubCategory' exists in df_rfm
    'Extended Price': 'sum',
    'QTY': 'sum',
    'Customer Number': 'first',  # Assuming each invoice has exactly one customer
    'Document Date': 'first',  # Date of the invoice
    'Location ID': 'first',  # Location of the invoice
    'SOP Number' : 'first'
}).rename(columns={
    'Item Number': 'Product Variety',
    'Category': 'Category Variety',
    'Subcategory': 'SubCategory Variety',
    'Extended Price': 'Total Extended Price',
    'QTY': 'Total QTY'
})

# Calculate Percentage Offered and Count of Distinct Offered Items
offered_data = df_invoice[df_invoice['Offer Status'] == 'Offer'].groupby('SOP Number').agg({
    'QTY': 'sum',
    'Item Number': pd.Series.nunique
}).rename(columns={
    'Item Number': 'Distinct Offered Items'
})

invoice_df = invoice_df.merge(offered_data, how='left', left_index=True, right_index=True)
invoice_df['Percentage Offered'] = invoice_df['QTY'] / invoice_df['Total QTY']
invoice_df.drop(columns='QTY', inplace=True)  # Remove the extra 'QTY' column

# Fill NaN values with appropriate defaults
invoice_df['Percentage Offered'].fillna(0, inplace=True)
invoice_df['Distinct Offered Items'].fillna(0, inplace=True)

# Save the aggregated data to a CSV
invoice_df.to_csv('invoice.csv', index=False)

print("Aggregated invoice data saved to 'invoice.csv'.")

Aggregated invoice data saved to 'invoice.csv'.


In [ ]:
# Define the function to calculate RFM values
# def calculate_rfm_up_to_date(customer_number, invoice_date, df):
#     # Ensure invoice_date is a pandas Timestamp object
#     invoice_date = pd.to_datetime(invoice_date)

#     # Filter records for the customer up to the invoice date
#     customer_data = df[(df['Customer Number'] == customer_number) &
#                        (df['Document Date'] < invoice_date)]

#     # Calculate Recency, Frequency, and Monetary
#     last_purchase = customer_data['Document Date'].max()
#     recency = (invoice_date - last_purchase).days if last_purchase else None
#     frequency = customer_data['SOP Number'].nunique()
#     monetary = customer_data['Extended Price'].sum()

#     return recency, frequency, monetary
    ##############################

In [32]:
import pandas as pd

# Convert 'Document Date' to datetime in the original DataFrame if it's not already
df_invoice['Document Date'] = pd.to_datetime(df_invoice['Document Date'])

# Ensure the DataFrame is sorted by 'Document Date'
df_invoice.sort_values(by='Document Date', inplace=True)



def calculate_rfm_up_to_date(customer_number, invoice_date, df):
    # Filter records for the customer up to the invoice date
    invoice_date = pd.to_datetime(invoice_date)
    customer_data = df[(df['Customer Number'] == customer_number) &
                       (df['Document Date'] < invoice_date)]

    # Calculate Recency
    last_purchase = customer_data['Document Date'].max()
    if pd.notnull(last_purchase):
        recency = (invoice_date - last_purchase).days
    else:
        # For new customers, use a large number to indicate no previous purchase
        recency = df['Document Date'].max() - df['Document Date'].min()
        recency = (recency.days if pd.notnull(recency) else 0) + 1

    # Calculate Frequency and Monetary
    frequency = customer_data['SOP Number'].nunique()
    monetary = customer_data['Extended Price'].sum()

    return recency, frequency, monetary
    ##############################

# Create the RFM columns in invoice_df
invoice_df['Invoice_R'] = None
invoice_df['Invoice_F'] = None
invoice_df['Invoice_M'] = None

# Calculate RFM values for each row in invoice_df
for index, row in invoice_df.iterrows():
    r, f, m = calculate_rfm_up_to_date(row['Customer Number'], row['Document Date'], df_invoice)
    invoice_df.at[index, 'Invoice_R'] = r
    invoice_df.at[index, 'Invoice_F'] = f
    invoice_df.at[index, 'Invoice_M'] = m

# Save the results
invoice_df.to_csv("invoice.csv", index=False)
print("RFM calculations are complete and saved to invoice.csv.")


RFM calculations are complete and saved to invoice.csv.


In [33]:
# Save the results
invoice_df.to_csv("invoice.csv", index=False)
print("RFM calculations are complete and saved to invoice.csv.")

RFM calculations are complete and saved to invoice.csv.


In [14]:
# import pandas as pd

# # Assuming df is your DataFrame and it has the columns 'Customer Number', 'SOP Number', 'Item Number', 'Extended Price', 'Offer Status', and 'Location ID'

# # Convert 'Document Date' to datetime if it's not already
# df['Document Date'] = pd.to_datetime(df['Document Date'])

# # Sort by 'Document Date' to ensure we're getting the first purchase when we group
# df.sort_values(by='Document Date', inplace=True)

# # Group by 'Customer Number' and get the first purchase
# first_purchase_df = df.groupby('Customer Number').first().reset_index()

# # For the "buys again" column, we need to see if there are more purchases beyond the first
# # We can do this by checking if there's more than one unique 'SOP Number' per customer
# buys_again = df.groupby('Customer Number')['SOP Number'].nunique() > 1

# # Now we'll create a DataFrame from this series and reset the index so it can be merged
# buys_again_df = buys_again.to_frame('Buys Again').reset_index()

# # Replace True/False with 1/0
# buys_again_df['Buys Again'] = buys_again_df['Buys Again'].astype(int)

# # Merge this information back into the first_purchase_df
# first_purchase_df = first_purchase_df.merge(buys_again_df, on='Customer Number', how='left')

# # To calculate the percentage offered, we need to know the total quantity and offered quantity per SOP Number
# total_qty_per_invoice = df.groupby('SOP Number')['QTY'].sum()
# offered_qty_per_invoice = df[df['Offer Status'] == 'Offer'].groupby('SOP Number')['QTY'].sum()

# # Calculate percentage offered and merge it back into first_purchase_df
# percentage_offered_df = (offered_qty_per_invoice / total_qty_per_invoice).to_frame('Percentage Offered').reset_index()
# first_purchase_df = first_purchase_df.merge(percentage_offered_df, on='SOP Number', how='left')

# # Fill NaN values in 'Percentage Offered' with 0 (assuming NaN means no offer was made)
# first_purchase_df['Percentage Offered'].fillna(0, inplace=True)

# # Select only the columns we want to keep
# first_purchase_df = first_purchase_df[['Customer Number', 'Document Date', 'Extended Price', 'Item Number', 'Percentage Offered', 'Location ID', 'Buys Again']]

# # Rename columns for clarity
# first_purchase_df.rename(columns={
#     'Document Date': 'First Purchase Date',
#     'Extended Price': 'First Invoice Value',
#     'Item Number': 'Distinct Products',
#     'Location ID': 'Purchase Location'
# }, inplace=True)

# # Now first_purchase_df is ready and contains all the information you wanted
# first_purchase_df.to_csv('first_purchase_data.csv', index=False)
# print("Aggregated first purchase data saved to 'first_purchase_data.csv'.")


Aggregated first purchase data saved to 'first_purchase_data.csv'.


In [ ]:
# # First, group by 'Customer Number' and identify the first SOP Number (Invoice) for each customer
# first_invoice_per_customer = df.groupby('Customer Number')['SOP Number'].first().reset_index()

# # Merge this information with the original df to filter out only the first purchases
# first_purchases = pd.merge(first_invoice_per_customer, df, on=['Customer Number', 'SOP Number'], how='left')

# # Now group by 'Customer Number' and count the distinct 'Item Number' for the first purchase
# distinct_products_first_purchase = first_purchases.groupby('Customer Number')['Item Number'].nunique().reset_index()

# # Merge this count back into first_purchase_df
# first_purchase_df = pd.merge(first_purchase_df, distinct_products_first_purchase, on='Customer Number', how='left')

# # Rename the columns for clarity
# first_purchase_df.rename(columns={
#     'Item Number': 'Distinct Products in First Invoice'
# }, inplace=True)

# # Continue with the rest of your processing...


In [19]:
import pandas as pd

# Load your DataFrame 'df' here

# Convert 'Document Date' to datetime if it's not already
df['Document Date'] = pd.to_datetime(df['Document Date'])

# Sort by 'Document Date' to ensure we're getting the first purchase when we group
df.sort_values(by='Document Date', inplace=True)

# Group by 'Customer Number' and get the first invoice (SOP Number) for each customer
first_invoice_per_customer = df.groupby('Customer Number')['SOP Number'].first().reset_index()

# Merge this information with df to get all rows related to the first purchases
first_purchases = pd.merge(first_invoice_per_customer, df, on=['Customer Number', 'SOP Number'], how='left')

# Now group by 'Customer Number' and count the distinct 'Item Number' for the first purchase
distinct_products_first_purchase = first_purchases.groupby('Customer Number')['Item Number'].nunique().reset_index()

# Aggregate other necessary information from the first purchase
first_purchase_agg = first_purchases.groupby('Customer Number').agg({
    'Document Date': 'first',
    'Extended Price': 'sum',
    'Location ID': 'first',
    'SOP Number': 'first'  # Keep the SOP Number
}).reset_index()

# Merge the distinct products count into first_purchase_agg
first_purchase_df = pd.merge(first_purchase_agg, distinct_products_first_purchase, on='Customer Number', how='left')

# Check if the customer buys again (more than one unique 'SOP Number')
buys_again = df.groupby('Customer Number')['SOP Number'].nunique() > 1
buys_again_df = buys_again.to_frame('Buys Again').reset_index()
buys_again_df['Buys Again'] = buys_again_df['Buys Again'].astype(int)
first_purchase_df = pd.merge(first_purchase_df, buys_again_df, on='Customer Number', how='left')

# Rename columns for clarity
first_purchase_df.rename(columns={
    'Item Number': 'Distinct Products in First Invoice'
}, inplace=True)

# To calculate the percentage offered, ensure 'SOP Number' is present in both dataframes
total_qty_per_invoice = df.groupby('SOP Number')['QTY'].sum()
offered_qty_per_invoice = df[df['Offer Status'] == 'Offer'].groupby('SOP Number')['QTY'].sum()
percentage_offered_df = (offered_qty_per_invoice / total_qty_per_invoice).to_frame('Percentage Offered').reset_index()
first_purchase_df = pd.merge(first_purchase_df, percentage_offered_df, on='SOP Number', how='left')
first_purchase_df['Percentage Offered'].fillna(0, inplace=True)

# Save the aggregated data
first_purchase_df.to_csv('first_purchase_data.csv', index=False)
print("Aggregated first purchase data saved to 'first_purchase_data.csv'.")

Aggregated first purchase data saved to 'first_purchase_data.csv'.


In [ ]:
library(dplyr)
library(tidyr)
library(lubridate)


df_invoice <- df


invoice_df <- df_invoice %>%
  group_by(SOP_Number) %>%
  summarise(
    Product_Variety = n_distinct(Item_Number),
    Category_Variety = n_distinct(Category),
    SubCategory_Variety = n_distinct(Subcategory),
    Total_Extended_Price = sum(Extended_Price),
    Total_QTY = sum(QTY),
    Customer_Number = first(Customer_Number),
    Document_Date = first(Document_Date),
    Location_ID = first(Location_ID),
    SOP_Number = first(SOP_Number)
  )

# Calculate Percentage Offered and Count of Distinct Offered Items
offered_data <- df_invoice %>%
  filter(Offer_Status == "Offer") %>%
  group_by(SOP_Number) %>%
  summarise(
    QTY = sum(QTY),
    Distinct_Offered_Items = n_distinct(Item_Number)
  )

# Merging data
invoice_df <- left_join(invoice_df, offered_data, by = "SOP_Number")
invoice_df <- invoice_df %>%
  mutate(Percentage_Offered = QTY / Total_QTY) %>%
  select(-QTY)

# Fill NaN values with appropriate defaults
invoice_df$Percentage_Offered[is.na(invoice_df$Percentage_Offered)] <- 0
invoice_df$Distinct_Offered_Items[is.na(invoice_df$Distinct_Offered_Items)] <- 0

# Save the aggregated data to a CSV
write.csv(invoice_df, 'invoice.csv', row.names = FALSE)
print("Aggregated invoice data saved to 'invoice.csv'.")


In [ ]:

df_invoice$Document_Date <- ymd(df_invoice$Document_Date)

# Ensure the DataFrame is sorted by 'Document Date'
df_invoice <- df_invoice %>% arrange(Document_Date)

calculate_rfm_up_to_date <- function(customer_number, invoice_date, df) {
  # Filter records for the customer up to the invoice date
  customer_data <- df %>% filter(Customer_Number == customer_number, Document_Date < invoice_date)

  # Calculate Recency
  last_purchase <- max(customer_data$Document_Date, na.rm = TRUE)
  if (!is.na(last_purchase)) {
    recency <- as.numeric(invoice_date - last_purchase)
  } else {
    # For new customers, use a large number to indicate no previous purchase
    recency_range <- max(df$Document_Date, na.rm = TRUE) - min(df$Document_Date, na.rm = TRUE)
    recency <- as.numeric(recency_range) + 1
  }

  # Calculate Frequency and Monetary
  frequency <- n_distinct(customer_data$SOP_Number)
  monetary <- sum(customer_data$Extended_Price, na.rm = TRUE)

  return(c(Recency = recency, Frequency = frequency, Monetary = monetary))
}

# Create the RFM columns in invoice_df
invoice_df$Invoice_R <- NA
invoice_df$Invoice_F <- NA
invoice_df$Invoice_M <- NA

# Calculate RFM values for each row in invoice_df
for (i in 1:nrow(invoice_df)) {
  rfm_values <- calculate_rfm_up_to_date(invoice_df$Customer_Number[i], invoice_df$Document_Date[i], df_invoice)
  invoice_df$Invoice_R[i] <- rfm_values["Recency"]
  invoice_df$Invoice_F[i] <- rfm_values["Frequency"]
  invoice_df$Invoice_M[i] <- rfm_values["Monetary"]
}

# Save the results
write.csv(invoice_df, "invoice.csv", row.names = FALSE)
print("RFM calculations are complete and saved to invoice.csv.")
